In [5]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser

In [6]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']

In [22]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection_job #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

In [8]:
!pip install dnspython

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

In [10]:

track = ['IAjob'] # define the keywords, tweets contain JMU

#locations = [-78.9326449,38.4150904,-78.8816972,38.4450731] #defin the location, in Harrisonburg, VA

In [23]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track)

1191807409049784320


KeyboardInterrupt: 

In [ ]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

In [ ]:

tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "jmu"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

In [ ]:
track = ['jmu'] # define the keywords, tweets contain JMU

locations = [-78.9326449,38.4150904,-78.8816972,38.4450731] #defin the location, in Harrisonburg, V

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track, locations = locations)

In [24]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [25]:

count = 100 #number of returned tweets, default and max is 100
#geocode = "38.4392897,-78.9412224,50mi"  # defin the location, in Harrisonburg, VA
q = "IA job"                               #define the keywords, tweets contain JMU

In [26]:
search_results = rest_api.search.tweets( count=count,q=q)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

'Tue Nov 05 19:56:34 +0000 2019'
'Tue Nov 05 19:55:54 +0000 2019'
'Tue Nov 05 19:55:13 +0000 2019'
'Tue Nov 05 19:54:54 +0000 2019'
'Tue Nov 05 19:54:43 +0000 2019'
'Tue Nov 05 19:50:34 +0000 2019'
'Tue Nov 05 19:50:24 +0000 2019'
'Tue Nov 05 19:49:57 +0000 2019'
'Tue Nov 05 19:48:43 +0000 2019'
'Tue Nov 05 19:46:52 +0000 2019'
'Tue Nov 05 19:46:43 +0000 2019'
'Tue Nov 05 19:46:18 +0000 2019'
'Tue Nov 05 19:45:33 +0000 2019'
'Tue Nov 05 19:45:13 +0000 2019'
'Tue Nov 05 19:44:52 +0000 2019'
'Tue Nov 05 19:44:23 +0000 2019'
'Tue Nov 05 19:42:21 +0000 2019'
'Tue Nov 05 19:39:12 +0000 2019'
'Tue Nov 05 19:38:32 +0000 2019'
'Tue Nov 05 19:37:22 +0000 2019'
'Tue Nov 05 19:30:41 +0000 2019'
'Tue Nov 05 19:29:21 +0000 2019'
'Tue Nov 05 19:28:53 +0000 2019'
'Tue Nov 05 19:28:22 +0000 2019'
'Tue Nov 05 19:27:20 +0000 2019'
'Tue Nov 05 19:25:30 +0000 2019'
'Tue Nov 05 19:24:02 +0000 2019'
'Tue Nov 05 19:23:37 +0000 2019'
'Tue Nov 05 19:20:40 +0000 2019'
'Tue Nov 05 19:19:50 +0000 2019'
'Tue Nov 0

In [27]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                         max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

'Tue Nov 05 17:02:58 +0000 2019'
'Tue Nov 05 17:01:56 +0000 2019'
'Tue Nov 05 17:00:49 +0000 2019'
'Tue Nov 05 16:59:55 +0000 2019'
'Tue Nov 05 16:59:35 +0000 2019'
'Tue Nov 05 16:58:06 +0000 2019'
'Tue Nov 05 16:57:05 +0000 2019'
'Tue Nov 05 16:55:54 +0000 2019'
'Tue Nov 05 16:50:54 +0000 2019'
'Tue Nov 05 16:47:05 +0000 2019'
'Tue Nov 05 16:46:21 +0000 2019'
'Tue Nov 05 16:40:08 +0000 2019'
'Tue Nov 05 16:37:14 +0000 2019'
'Tue Nov 05 16:33:52 +0000 2019'
'Tue Nov 05 16:33:33 +0000 2019'
'Tue Nov 05 16:33:25 +0000 2019'
'Tue Nov 05 16:32:42 +0000 2019'
'Tue Nov 05 16:32:15 +0000 2019'
'Tue Nov 05 16:25:57 +0000 2019'
'Tue Nov 05 16:25:02 +0000 2019'
'Tue Nov 05 16:24:01 +0000 2019'
'Tue Nov 05 16:23:22 +0000 2019'
'Tue Nov 05 16:22:02 +0000 2019'
'Tue Nov 05 16:19:42 +0000 2019'
'Tue Nov 05 16:15:20 +0000 2019'
'Tue Nov 05 16:11:51 +0000 2019'
'Tue Nov 05 16:11:50 +0000 2019'
'Tue Nov 05 16:09:00 +0000 2019'
'Tue Nov 05 16:05:29 +0000 2019'
'Tue Nov 05 16:02:32 +0000 2019'
'Tue Nov 0

'Mon Nov 04 21:15:52 +0000 2019'
'Mon Nov 04 21:15:15 +0000 2019'
'Mon Nov 04 21:13:22 +0000 2019'
'Mon Nov 04 21:12:57 +0000 2019'
'Mon Nov 04 21:11:47 +0000 2019'
'Mon Nov 04 21:10:04 +0000 2019'
'Mon Nov 04 21:09:29 +0000 2019'
'Mon Nov 04 21:02:22 +0000 2019'
'Mon Nov 04 21:02:11 +0000 2019'
'Mon Nov 04 21:00:01 +0000 2019'
'Mon Nov 04 20:58:01 +0000 2019'
'Mon Nov 04 20:57:51 +0000 2019'
'Mon Nov 04 20:57:32 +0000 2019'
'Mon Nov 04 20:57:11 +0000 2019'
'Mon Nov 04 20:56:50 +0000 2019'
'Mon Nov 04 20:55:13 +0000 2019'
'Mon Nov 04 20:54:13 +0000 2019'
'Mon Nov 04 20:53:09 +0000 2019'
'Mon Nov 04 20:52:50 +0000 2019'
'Mon Nov 04 20:51:51 +0000 2019'
'Mon Nov 04 20:49:19 +0000 2019'
'Mon Nov 04 20:46:39 +0000 2019'
'Mon Nov 04 20:45:48 +0000 2019'
'Mon Nov 04 20:45:19 +0000 2019'
'Mon Nov 04 20:39:58 +0000 2019'
'Mon Nov 04 20:39:41 +0000 2019'
'Mon Nov 04 20:37:29 +0000 2019'
'Mon Nov 04 20:36:38 +0000 2019'
'Mon Nov 04 20:35:13 +0000 2019'
'Mon Nov 04 20:34:38 +0000 2019'
'Mon Nov 0

KeyboardInterrupt: 

In [28]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

596
291


In [30]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "data mining"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: TMJ-USA IT-DB Jobs
text: Interested in a job in #WestDesMoines, IA? This could be a great fit. Click the link in our bio to apply: Data Repo… https://t.co/P6Szjl4A8n
----
name: TMJ-IAD Jobs
text: Cognizant is looking for teammates like you. See our latest #BusinessMgmt job openings, including "Data Entry Clerk… https://t.co/5WHRWYM6j0
----
name: TMJ-IA IT Jobs
text: Join the Robert Half Technology team! See our latest #IT job openings, including "Data/Information Architect", via… https://t.co/z19XWUlv9k
----
name: Crédit Agricole TS
text: 💡📣 Salon #Datajob Paris: Rendez-vous le 22 novembreà  @datajobparis, la rencontre annuelle des métiers de la #data… https://t.co/cFoGgcil1k
----
name: Crédit Agricole RH
text: RT @CA_TS_: 💡📣 Salon #Datajob Paris: Rendez-vous le 22 novembreà  @datajobparis, la rencontre annuelle des métiers de la #data et retrouvez…
----
name: Christian Dominé
text: RT @CA_TS_: 💡📣 Salon #Datajob Paris: Rendez-vous le 22 novembreà  @datajobparis, la rencontre